# Background / Antecedentes

**EN:**

The goal of this notebook is build a map similar to that published in the NYT by [Lauren Leatherby and Charlie Smart on July 2, 2020](https://www.nytimes.com/interactive/2020/07/02/us/coronavirus-cases-increase.html). It shows the evolution of COVID cases in Mexico across states and counties from Feb 2020 through early July 2020. Data is computed to present number of cases per day (CASOS) , on a rolling sum over the past 2 weeks (CASOS_14d), and it's equivalent measures for every 1000 people in each location (i.e. PER_1000).

This is for informational and visual purposes, and doesn't imply anything about the direction of the epidemic.

**ES:**

El objetivo de este cuaderno es construir un mapa similar al publicado en el NYT por [Lauren Leatherby y Charlie Smart el 2 de julio de 2020](https://www.nytimes.com/interactive/2020/07/02/us/coronavirus-cases-aumentar.html). El mapa muestra como los casos de COVID en México han evolucionado en cada estado y municipio desde febrero del 2020 hasta principios de julio del 2020. Los datos presentan el número casos por día (CASOS), la suma de casos durante últimos 14 días (CASOS_14d), y equivalentemente contiene medidas por cada 1000 habitantes en cada localidad (e.j. PER_1000).

Este mapa es para fines informativos y visuales, y no busca demostrar tendencias de casos y/o la epidemia.





In [1]:
import pandas as pd
import numpy as np
import os

from keplergl import KeplerGl


# Data / Datos

Source/ Fuente: 
- COVID: https://www.gob.mx/salud/documentos/datos-abiertos-152127
- INEGI: https://www.inegi.org.mx/app/ageeml/#


In [4]:
#! unzip "data/AGEEML_2020751524435.csv.zip" -d data/
! unzip "data/datos_abiertos_covid19_12072020.zip" -d data/
#! unzip "data/diccionario_datos_covid19.zip" -d data/

Archive:  data/datos_abiertos_covid19_12072020.zip
  inflating: data/200712COVID19MEXICO.csv  


In [5]:
# Import full covid data set from datos abiertos 
# https://www.gob.mx/salud/documentos/datos-abiertos-152127

covid_mx = pd.read_csv('data/200712COVID19MEXICO.csv', engine='python')


In [6]:
covid_mx.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,2020-07-12,1db186,2,3,15,1,15,15,76,1,...,2,1,2,2,1,1,99,México,99,97
1,2020-07-12,19b7a9,2,3,27,2,27,27,13,1,...,2,2,2,2,1,1,99,México,99,97
2,2020-07-12,1ae47a,2,4,9,1,14,9,5,1,...,2,2,2,2,99,1,99,México,99,97
3,2020-07-12,0e345e,2,4,30,1,30,30,193,1,...,2,2,2,2,99,1,99,México,99,97
4,2020-07-12,170aea,2,4,9,2,15,15,104,1,...,2,2,2,1,99,1,99,México,99,97


# Initial filtering / Filtros Iniciales

**EN:**

Here I subset for certain columns, convert variables to appropriate data types and keep only valid states and counties (i.e. codes based on data dictionary that aren't for unknown or NA entries)

Most data contains valid entries so this doesn't represent issues when visualizing data.


**ES:**

Aquí tomo ciertas columnas, transformo variables y mantengo solo estados y municipios con códigos que son válidos (es decir, códigos que representan estados/municipios reales y no datos faltantes)

La mayoría de los datos contienen códigos reales por lo que esta face no representa problemas al visualizar datos.



In [7]:
# Get subset of columns
covid_mx_sub = covid_mx.loc[:,['FECHA_SINTOMAS', 'ID_REGISTRO', 'ENTIDAD_RES', 'MUNICIPIO_RES']]
covid_mx_sub['FECHA_SINTOMAS'] = pd.to_datetime(covid_mx_sub['FECHA_SINTOMAS'])
covid_mx_sub['CASOS'] = covid_mx_sub.ID_REGISTRO
covid_mx_sub.set_index('FECHA_SINTOMAS', inplace=True)


In [8]:
covid_mx_sub.head()

,ID_REGISTRO,ENTIDAD_RES,MUNICIPIO_RES,CASOS
FECHA_SINTOMAS,,,,
2020-05-11,1db186,15,76,1db186
2020-06-11,19b7a9,27,13,19b7a9
2020-03-28,1ae47a,9,5,1ae47a
2020-04-12,0e345e,30,193,0e345e
2020-04-08,170aea,15,104,170aea


In [9]:
# Filtering
covid_mx_sub = covid_mx_sub[covid_mx_sub.MUNICIPIO_RES <900]
covid_mx_sub = covid_mx_sub.loc[:'2020-07-12']

# Rolling sum / Agregando cifras

**EN:**

Here I create the two week rolling sum counts for cases as in the NYT map


**ES:**

Aquí creo los recuentos de casos acumulados cada dos semanas como en el mapa del NYT



In [10]:
#  Create roling sum using agg and rolling methods

covid_agg = covid_mx_sub.groupby(['FECHA_SINTOMAS','ENTIDAD_RES', 'MUNICIPIO_RES']).agg({'CASOS':'count', 'ID_REGISTRO':'first'})

rolling_14d = covid_agg.reset_index(level=[1,2]).groupby(['ENTIDAD_RES', 'MUNICIPIO_RES'])[['CASOS']].rolling('14d').sum().reset_index()
rolling_14d.rename(columns={"CASOS": "CASOS_14d"},inplace=True)

covid_agg.reset_index(inplace=True)

In [11]:
# Merge to create dataframe in format is skeleton of final dataframe 

covid_agg_roll_14d = covid_agg.merge(rolling_14d, on=['FECHA_SINTOMAS', 'ENTIDAD_RES', 'MUNICIPIO_RES'])

# Data Dictionary (States & Counties) / Diccionario de Datos (Estados y Municipios)

**EN:**

Here I get dataframes with merged states and counties based on state/county codes


**ES:**

Aquí obtengo unas tablas con combinaciones de estados y municipios en base a los códigos unicos de estado / municipio


In [12]:
# Get data dictionary of states (estados) and counties (municipios)

state_data_dictionary = pd.read_excel('data/diccionario_datos_covid19/Catalogos_0412.xlsx', 
                                      sheet_name=['Catálogo de ENTIDADES', 'Catálogo MUNICIPIOS'])

states = state_data_dictionary['Catálogo de ENTIDADES']
counties = state_data_dictionary['Catálogo MUNICIPIOS']

In [13]:
# Filter valid counties and states

valid_counties = counties[counties.CLAVE_MUNICIPIO <900]
valid_states = states[states.CLAVE_ENTIDAD < 90]

In [14]:
# Combine states and counties
valid_state_counties = valid_counties.merge(valid_states, on='CLAVE_ENTIDAD')

In [12]:
valid_state_counties.head()

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,ABREVIATURA
0,1,AGUASCALIENTES,1,AGUASCALIENTES,AS
1,2,ASIENTOS,1,AGUASCALIENTES,AS
2,3,CALVILLO,1,AGUASCALIENTES,AS
3,4,COSÍO,1,AGUASCALIENTES,AS
4,5,JESÚS MARÍA,1,AGUASCALIENTES,AS


## State/County Coordinates + Population | Coordenadas de Estados/Municipios  + Población 

**EN:**

Using INEGI data, I get state and county unique codes to then merge these with the previous dataframe


**ES:**

Usando datos del INEGI obtuve códigos únicos para estado y municipio que luego uso para combinar datos con la tabla anterior

In [17]:

# Helper function in case a new INEGI file needs to get downloaded and formatted
def format_INEGI_data(coord_file, valid_state_counties):
    '''Helper function to wrangle raw INEGI coordinate data from https://www.inegi.org.mx/app/ageeml/#
       into a pandas DataFrame that can be merged with MX government COVID data 
       (https://www.gob.mx/salud/documentos/datos-abiertos-152127)
       
       Inputs:
       - coord_file |str : file path to INEGI 
       
       - valid_state_counties | obj : DataFrame with state and county codes found from COVID gov't data
       
       Output | obj : dataframe 
       
       Dataframe columns: 'CLAVE_MUNICIPIO', 'MUNICIPIO', 'CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA',
       'Lat_Decimal', 'Lon_Decimal', 'Pob_Total' '''
    
    # Get lat, long data from https://www.inegi.org.mx/app/ageeml/# data
    coords = pd.read_csv(coord_file)

    coords_filt = coords.drop_duplicates(subset=['Cve_Ent', 'Cve_Mun'], keep='first')
    coords_filt = coords_filt[['Cve_Ent', 'Cve_Mun', 'Lat_Decimal', 'Lon_Decimal', 'Pob_Total']]
    
    # Merge with dataframe of states/counties found in COVID data
    valid_states_counties_w_coords = valid_state_counties.merge(coords_filt, 
                                                            left_on=['CLAVE_ENTIDAD', 'CLAVE_MUNICIPIO'],
                                                            right_on=['Cve_Ent', 'Cve_Mun'])
    
    
    # Select only necessary columns (i.e. keep everything except those in col_idx)
    col_idx = valid_states_counties_w_coords.columns.isin(['ABREVIATURA', 'Cve_Ent', 'Cve_Mun'])
    valid_states_counties_w_coords = valid_states_counties_w_coords.iloc[:,~col_idx]
    
    # Save file
    valid_states_counties_w_coords.to_csv("data/estados_municipios_AGEEML.csv", 
                                          index=False)
    
    

# If existing file exists then load, otherwise write and load
if os.path.isfile("data/estados_municipios_AGEEML.csv"):
    valid_states_counties_w_coords = pd.read_csv('data/estados_municipios_AGEEML.csv')
else:
    format_INEGI_data('data/AGEEML_2020751524435.csv', valid_state_counties)
    valid_states_counties_w_coords = pd.read_csv('data/estados_municipios_AGEEML.csv')

In [18]:
valid_states_counties_w_coords.head()

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD,ENTIDAD_FEDERATIVA,Lat_Decimal,Lon_Decimal,Pob_Total
0,1,AGUASCALIENTES,1,AGUASCALIENTES,21.879823,-102.296047,722250
1,2,ASIENTOS,1,AGUASCALIENTES,22.238317,-102.089275,4517
2,3,CALVILLO,1,AGUASCALIENTES,21.846907,-102.718751,19742
3,4,COSÍO,1,AGUASCALIENTES,22.366409,-102.300044,4898
4,5,JESÚS MARÍA,1,AGUASCALIENTES,21.961273,-102.343416,43012


## Create main dataframe / Crear tabla principal

**EN:**

Merge COVID case data with state/county population + coordinates

**ES:**

Juntar datos de casos de COVID + datos del INEGI


In [19]:
# 7 rows lost in merge from initial covid_mx_sub 
# those are from INEGI municipios codes not found in the states_counties, but yes in covid data 
# municipal codes are potential bad entries

df_covid = covid_agg_roll_14d.merge(valid_states_counties_w_coords, how='inner',
                                    left_on=['ENTIDAD_RES', 'MUNICIPIO_RES'],
                                    right_on=['CLAVE_ENTIDAD', 'CLAVE_MUNICIPIO'])

df_covid = df_covid.iloc[:, ~df_covid.columns.isin(['ENTIDAD_RES', 'MUNICIPIO_RES', 
                                                    'CLAVE_MUNICIPIO', 'CLAVE_ENTIDAD'])]

In [37]:
df_covid.head()

,FECHA_SINTOMAS,CASOS,ID_REGISTRO,CASOS_14d,MUNICIPIO,ENTIDAD_FEDERATIVA,Lat_Decimal,Lon_Decimal,Pob_Total
0,2020-01-01,1,18476a,1.0,MONCLOVA,COAHUILA DE ZARAGOZA,26.901242,-101.417224,215271
1,2020-02-18,1,155459,1.0,MONCLOVA,COAHUILA DE ZARAGOZA,26.901242,-101.417224,215271
2,2020-03-01,2,0eec1a,3.0,MONCLOVA,COAHUILA DE ZARAGOZA,26.901242,-101.417224,215271
3,2020-03-04,1,17cc47,3.0,MONCLOVA,COAHUILA DE ZARAGOZA,26.901242,-101.417224,215271
4,2020-03-08,2,0f6b35,5.0,MONCLOVA,COAHUILA DE ZARAGOZA,26.901242,-101.417224,215271


## Generate map / Generar mapa

**EN:**

Prepare final dataframe and use KeplerGl to generate map (use pre-configured hex_config.py file)


**ES:**

Terminar de prepar los datos para el mapa de KeplerGl


In [23]:
# Get data up to July 3rd, 2020
df_covid_mid_feb = df_covid.set_index('FECHA_SINTOMAS').loc[:'2020-07-12']
df_covid_mid_feb.reset_index(inplace=True)

# Turn datetime to strings for map
df_covid_mid_feb['FECHA_SINTOMAS'] = df_covid_mid_feb.FECHA_SINTOMAS.dt.strftime('%Y-%m-%d %H:%M:%S')

# Per 1000 values for rolling sum and daily cases
df_covid_mid_feb['14d_PER_1000'] = df_covid_mid_feb.CASOS_14d.values / df_covid_mid_feb.Pob_Total.values.astype(float)*1000
df_covid_mid_feb['PER_1000'] = df_covid_mid_feb.CASOS.values / df_covid_mid_feb.Pob_Total.values.astype(float)*1000


In [24]:
# run configuration file
%run hex_config.py

In [25]:
# Create and save map
map_1 = KeplerGl(height=500, data={'covid_mx': df_covid_mid_feb}, config=config)
map_1.save_to_html(file_name="covid_mx.html")

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to covid_mx.html!
